In [48]:
import pandas as pd
import numpy as np
dataframe=pd.read_csv('result/nid/location_dectection_seul.csv')
dataframe=dataframe.drop_duplicates(subset=['time','id'],keep='first')
dataframe.head(20)

,time,id,x,y,w,h,center_x,center_y
0,4,0,978,1013,34,22,995,1024
1,4,1,319,918,58,32,348,934
2,4,2,188,890,25,51,200,915
5,4,3,750,827,41,54,770,854
6,4,4,375,806,49,21,399,816
8,4,5,666,645,42,30,687,660
9,4,6,330,636,48,37,354,654
11,4,7,723,603,31,38,738,622
13,4,8,1025,574,45,37,1047,592
14,4,9,406,566,116,16,464,574


In [49]:
min_time=min(dataframe['time'])
max_time=max(dataframe['time'])

In [121]:
#init_time=min(dataframe[dataframe['id']==id]['time'])
def find_fourmis(dataframe,min_time,max_time,id,time):
    init_time=time
    #print('init_time=',init_time)
    dataframe_temp=dataframe[(dataframe['time']<time+15) & (dataframe['time']>=time)].drop_duplicates(subset=['id'],keep='last')
    x,y,time=dataframe_temp[dataframe_temp['id']==id][['center_x','center_y','time']].values[0]
    pos=np.array([x,y])
    #print('pos=',pos,'time=',time)
    if time>max_time-15:
        return dataframe
    if time>init_time:
        #print('found the same id')
        find_fourmis(dataframe,min_time,max_time,id,time)
    else: # if not, find the probable candidate
        #print('not found the same id')
        for i in range (len(dataframe_temp)):
            temp_x,temp_y,temp_time=dataframe_temp.iloc[i][['center_x','center_y','time']].values
            temp_pos=np.array([temp_x,temp_y])
            if np.linalg.norm(temp_pos-pos)<(temp_time-time)*5:
                id_ans=dataframe_temp.iloc[i]['id']
                time=temp_time
                #print('candidate found at',time)
                # change the id of the candidate
                dataframe_temp.loc[dataframe_temp[dataframe_temp['id']==id_ans].index,'id']=id
                dataframe.loc[dataframe[dataframe['id']==id_ans].index,'id']=id
                find_fourmis(dataframe,min_time,max_time,id,time)
                #print('found at',time)
    #print('perdu a',time)
    return dataframe
    
        
        


In [131]:
len_list=dataframe['id'].unique()
new_dataframe=pd.DataFrame(columns=['id','center_x','center_y','time'])
for id in len_list:
    if id in dataframe['id'].unique():
        time= min(dataframe[dataframe['id']==id]['time'])
        dataframe=find_fourmis(dataframe,min_time,max_time,id,time)
        if len(dataframe[dataframe['id']==id])>20:
            # print('------------------------------')
            # print('id=',id)
            # print(dataframe[dataframe['id']==id][:5])
            # print(dataframe[dataframe['id']==id][-5:])
            # print('------------------------------')
            frames=[new_dataframe,dataframe[dataframe['id']==id]]
            new_dataframe=pd.concat(frames)

In [144]:
new_dataframe=new_dataframe.drop_duplicates(subset=['time','id'],keep='last')

In [145]:
fourmis_display=new_dataframe.sort_values(by=['time'])
fourmis_display.to_csv('result/nid/location_dectection_diplay.csv',index=False)

In [146]:
fourmis_display

,id,center_x,center_y,time,x,y,w,h
9,6,354,654,4,330.0,636.0,48.0,37.0
25,6,356,652,5,334.0,636.0,44.0,32.0
22,16,679,1026,5,637.0,1019.0,85.0,14.0
28,6,286,748,6,273.0,725.0,27.0,47.0
27,16,603,1033,6,573.0,1021.0,60.0,24.0
...,...,...,...,...,...,...,...,...
5915,374,520,605,491,498.0,587.0,44.0,37.0
5907,371,880,917,491,858.0,907.0,44.0,20.0
5917,360,79,513,491,65.0,481.0,28.0,64.0
5914,313,715,622,491,704.0,607.0,22.0,31.0


In [ ]:
import 

In [156]:
traces=[]
for id in fourmis_display['id'].unique():
    traces.append([dataframe[dataframe['id']==id]['center_x'].values,dataframe[dataframe['id']==id]['center_y'].values])
traces[0]

[array([354, 200, 356, 286, 281, 286, 287, 293, 287, 296, 264, 263, 263,
        263, 263, 292, 297, 294, 294, 265, 266, 295, 295, 288, 298, 289,
        288, 287, 288, 290, 290, 273, 265, 264, 265, 264, 264, 264, 263,
        263, 263, 264, 265, 264]),
 array([654, 734, 652, 748, 750, 657, 657, 657, 656, 657, 602, 602, 603,
        603, 603, 653, 653, 652, 651, 602, 602, 650, 649, 663, 653, 664,
        665, 663, 663, 662, 663, 604, 605, 606, 606, 606, 608, 609, 609,
        610, 610, 610, 610, 613])]

In [158]:
traces=np.array(traces)
traces.shape

/var/folders/2r/9qy6_yrj3hz_h1xfq66mhfl80000gn/T/ipykernel_16117/1166321531.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  traces=np.array(traces)


(51, 2)

In [159]:
len=5
x,y,h,w=fourmis_display[fourmis_display['time']==len]['x'].values,fourmis_display[fourmis_display['time']==len]['y'],fourmis_display[fourmis_display['time']==len]['h'],fourmis_display[fourmis_display['time']==len]['w']

In [166]:
x.shape

(2,)

In [136]:
pts = np.array([[25, 70], [25, 160],
                [110, 200], [200, 160],
                [200, 70], [110, 20]],
               np.int32)
 
pts = pts.reshape((-1, 1, 2))
 
pts
 

array([[[ 25,  70]],

       [[ 25, 160]],

       [[110, 200]],

       [[200, 160]],

       [[200,  70]],

       [[110,  20]]], dtype=int32)